In [2]:
import plotly.graph_objects as go
import pandas as pd

import plotly.express as px

import plotly.offline as offline

from ipywidgets import widgets

from datetime import datetime


In [3]:
data= pd.read_csv("assets/bond_details.csv", sep=",", parse_dates=["Date_of_Purchase"])
data["Denomination"] = data["Denomination"].astype(float)

In [4]:

data['Month_Year'] = data['Date_of_Purchase'].dt.strftime('%b-%Y')
#data['Month_Year'] = data['Date_of_Purchase'].dt.to_period('m')
#data['Month_Year'] = pd.to_datetime(data['Month_Year'], format='%b-%Y')

In [5]:
## Normal overall analysis of the bonds.
# We group data by purchasers name.

bonds_tree= data.groupby("Purchaser_Name")["Denomination"].apply(sum).to_dict()  # only use it for a combined analysis of all bonds
# We can use the following code to create a treemap of the bonds.         
fig=go.Figure(go.Treemap(labels= list(bonds_tree.keys()), parents= [""]*len(bonds_tree.keys()), values= list(bonds_tree.values()), text= list(bonds_tree.keys()), marker_colors=px.colors.qualitative.Plotly, textinfo="label+text+value"))    
fig.update_layout(width=750, height=750, margin=dict(l=0, r=0, t=0, b=0))
    
offline.plot(fig, filename="ipy_bonds_map.html", auto_open=True)

'ipy_bonds_map.html'

In [6]:
# Sample DataFrame
#datetime.datetime.strptime(year.value[0], "%b-%Y")

# Function to update the treemap
def update_treemap(y):
    year = selection_range_slider.value

    idx_start = data.index[data['Month_Year'] == year[0]][0]
    idx_end = data.index[data['Month_Year'] == year[1]][-1]

    filtered_df = data.loc[idx_start : idx_end]
    #filtered_df= data.between_time(parser.isoparse(year.value[0]), parser.isoparse(year.value[1]))
    #filtered_df = data.loc[data['Month_Year'] >= datetime.strptime(year.value[0], "%b-%Y") & data['Month_Year'] <= datetime.strptime(year.value[1], "%b-%Y")]
    #filtered_df = data.loc[(datetime.strptime(data['Month_Year'],"%b-%Y") >= datetime.strptime(year.value[0], "%b-%Y")) & (datetime.strptime(data['Month_Year'],"%b-%Y") <= datetime.strptime(year.value[1], "%b-%Y"))]
    dict_b= filtered_df.groupby("Purchaser_Name")["Denomination"].apply(sum).to_dict()
    fig = go.Figure(go.Treemap(
        labels = list(dict_b.keys()),
        parents = [""]*len(dict_b.keys()),   #[""]*len(bonds_tree.keys()
        values = list(dict_b.values()),
        textinfo = "label+value"
    ))
    fig.update_layout( width=1000 , height= 1000 , margin = dict(t=25, l=25, r=25, b=25))
    return fig

# Slider widget for year selection
selection_range_slider = widgets.SelectionRangeSlider(
    options=unique_dates,
    index=(0, len(unique_dates)-1),
    start=0,
    end=len(unique_dates)-1,
    #value=('Apr-2019', 'Oct-2021'),
    step=1,
    description='Dates',
    orientation='horizontal',
    layout={'width': '1000px'},
    continuous_update=False,
)

#widgets.interactive(update_treemap, y=selection_range_slider)

widgets.interact(update_treemap, y=selection_range_slider)
#widgets.widget_layout.Layout(display='flex', justify_content='space-between', align_items='center', width='30%')
#embed_minimal_html('export.html', views=[update_treemap, selection_range_slider], title='Widgets export')

NameError: name 'unique_dates' is not defined

In [7]:
data2= pd.read_csv("assets/party_details.csv", sep=",", parse_dates=["Date_of_Encashment"])
data2["Denomination"] = data2["Denomination"].astype(float)

In [8]:
data2['Month_Year'] = data2['Date_of_Encashment'].dt.strftime('%b-%Y')

In [9]:
data.Denomination.unique()

array([1.e+05, 1.e+06, 1.e+07, 1.e+03, 1.e+04])

In [10]:

date_dict= data.groupby("Month_Year")["Denomination"].size().to_dict()
date_dict

{'Apr-2019': 1335,
 'Apr-2021': 974,
 'Apr-2022': 811,
 'Apr-2023': 1470,
 'Dec-2022': 260,
 'Jan-2020': 139,
 'Jan-2021': 151,
 'Jan-2022': 2068,
 'Jan-2023': 437,
 'Jan-2024': 897,
 'Jul-2019': 101,
 'Jul-2021': 465,
 'Jul-2022': 480,
 'Jul-2023': 1371,
 'May-2019': 1187,
 'Nov-2022': 954,
 'Nov-2023': 1109,
 'Oct-2019': 531,
 'Oct-2020': 321,
 'Oct-2021': 1057,
 'Oct-2022': 741,
 'Oct-2023': 2012}

In [11]:
#filtered_df = data.loc[data['Month_Year'] == 'Apr-2019']
df_grouped = data.groupby(['Month_Year', 'Denomination']).size().unstack(fill_value=0)

In [12]:
data['dt_month_year'] = data['Date_of_Purchase'].dt.to_period('M')

In [13]:

# Normalize counts to percentages
df_grouped_percent = df_grouped.div(df_grouped.sum(axis=1), axis=0) * 100

# Create stacked area chart
trace1 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[1000],
    mode='lines',
    stackgroup='one',
    name='1 Thousand'
)
trace2 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[10000],
    mode='lines',
    stackgroup='one',
    name='10 Thousand'
)
trace3 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[100000],
    mode='lines',
    stackgroup='one',
    name='1 Lakh'
)
trace4 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[1000000],
    mode='lines',
    stackgroup='one',
    name='10 Lakh'
)
trace5 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[10000000],
    mode='lines',
    stackgroup='one',
    name='1 Crore'
)

data = [trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title='Type of Denominations (Percentage)',
    xaxis=dict(title='Month Year'),
    yaxis=dict(title='Percentage'),
    hovermode='x'
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [14]:
data2

,Date_of_Encashment,Name_of_Party,Denomination,Month_Year
0,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
1,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
2,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,10000000.0,Apr-2019
3,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
4,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
...,...,...,...,...
20416,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20417,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20418,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20419,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024


In [15]:
dict_x= data.groupby("Purchaser_Name")["Denomination"].apply(sum).to_dict()

AttributeError: 'list' object has no attribute 'groupby'

In [16]:
dict_x

NameError: name 'dict_x' is not defined

In [17]:
# Create a histogram trace
histogram_trace = go.Histogram(x=list(dict_x.values()), xbins=dict(start=1000, end=10000000, size=1000000), marker=dict(color='blue'))

# Create a layout
layout = go.Layout(
    title='Histogram of Values from Dictionary',
    xaxis=dict(title='Values'),
    yaxis=dict(title='Frequency')
)

# Create a figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the figure
fig.show()


NameError: name 'dict_x' is not defined

In [18]:
sorted_data = dict(sorted(dict_x.items(), key=lambda item: item[1]))


NameError: name 'dict_x' is not defined

In [19]:
dict(list(sorted_data.items())[-20:])

NameError: name 'sorted_data' is not defined

In [20]:
parties = pd.read_csv('assets/parties.csv', parse_dates=['Date_of_Encashment'])
purchasers = pd.read_csv('assets/purchasers.csv', parse_dates=['Date_of_Purchase'])

df = pd.merge(purchasers,parties,how='left',left_on='Bond_number',right_on='Bond_number').dropna(axis=0)


In [21]:
df

,Sr_No._x,Reference_No_(URN),Journal_Date,Date_of_Purchase,Date_of_Expiry,Name_of_Purchaser,Prefix_x,Bond_Number_x,Denominations_x,Issue_Branch_Code,...,Bond_number,Sr_No._y,Date_of_Encashment,Name_of_Political_Party,Account_no.,Prefix_y,Bond_Number_y,Denominations_y,Pay_Branch_Code,Pay_Teller
0,1,00001201904120000001166,4/12/2019 0:00,2019-04-12,4/26/2019 0:00,A B C INDIA LIMITED,TL,11448,1000000,1,...,TL11448,2766.0,2019-04-25,BHARATIYA JANATA PARTY,*******8244,TL,11448.0,1000000.0,691.0,3300196.0
1,2,00001201904120000001166,4/12/2019 0:00,2019-04-12,4/26/2019 0:00,A B C INDIA LIMITED,TL,11447,1000000,1,...,TL11447,2767.0,2019-04-25,BHARATIYA JANATA PARTY,*******8244,TL,11447.0,1000000.0,691.0,3300196.0
2,3,00001201904120000001166,4/12/2019 0:00,2019-04-12,4/26/2019 0:00,A B C INDIA LIMITED,TL,11441,1000000,1,...,TL11441,2768.0,2019-04-25,BHARATIYA JANATA PARTY,*******8244,TL,11441.0,1000000.0,691.0,3300196.0
3,4,00001201904120000001166,4/12/2019 0:00,2019-04-12,4/26/2019 0:00,A B C INDIA LIMITED,OL,1113,100000,1,...,OL1113,2807.0,2019-04-25,BHARATIYA JANATA PARTY,*******8244,OL,1113.0,100000.0,691.0,3300196.0
4,5,00001201904120000001166,4/12/2019 0:00,2019-04-12,4/26/2019 0:00,A B C INDIA LIMITED,OL,1118,100000,1,...,OL1118,2808.0,2019-04-25,BHARATIYA JANATA PARTY,*******8244,OL,1118.0,100000.0,691.0,3300196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18866,18867,00001202401110000003729,1/11/2024 0:00,2024-01-11,1/25/2024 0:00,WESTWELL GASES PRIVATE LIMITED,OC,15695,10000000,1,...,OC15695,20400.0,2024-01-23,DRAVIDA MUNNETRA KAZHAGAM (DMK),*******8935,OC,15695.0,10000000.0,800.0,1011942.0
18867,18868,00001202401110000003729,1/11/2024 0:00,2024-01-11,1/25/2024 0:00,WESTWELL GASES PRIVATE LIMITED,OC,15693,10000000,1,...,OC15693,20401.0,2024-01-23,DRAVIDA MUNNETRA KAZHAGAM (DMK),*******8935,OC,15693.0,10000000.0,800.0,1011942.0
18868,18869,00001202401110000003729,1/11/2024 0:00,2024-01-11,1/25/2024 0:00,WESTWELL GASES PRIVATE LIMITED,OC,15697,10000000,1,...,OC15697,20402.0,2024-01-23,DRAVIDA MUNNETRA KAZHAGAM (DMK),*******8935,OC,15697.0,10000000.0,800.0,1011942.0
18869,18870,00001202401110000003729,1/11/2024 0:00,2024-01-11,1/25/2024 0:00,WESTWELL GASES PRIVATE LIMITED,OC,17803,10000000,1,...,OC17803,20403.0,2024-01-23,DRAVIDA MUNNETRA KAZHAGAM (DMK),*******8935,OC,17803.0,10000000.0,800.0,1011942.0


In [92]:
filtered_df.dtypes

Sr_No._x                            int64
Reference_No_(URN)                 object
Journal_Date                       object
Date_of_Purchase           datetime64[ns]
Date_of_Expiry                     object
Name_of_Purchaser                  object
Prefix_x                           object
Bond_Number_x                       int64
Denominations_x                     int64
Issue_Branch_Code                   int64
Issue_Teller                        int64
Status                             object
Bond_number                        object
Sr_No._y                          float64
Date_of_Encashment         datetime64[ns]
Name_of_Political_Party            object
Account_no.                        object
Prefix_y                           object
Bond_Number_y                     float64
Denominations_y                   float64
Pay_Branch_Code                   float64
Pay_Teller                        float64
dtype: object

In [65]:
filtered_df = df.loc[df['Date_of_Purchase'] < 'Feb-2024']

In [66]:
filtered_df_groups= filtered_df.groupby(["Name_of_Purchaser", "Name_of_Political_Party"])["Denominations_x"].apply(sum).reset_index()
filtered_df_groups

,Name_of_Purchaser,Name_of_Political_Party,Denominations_x
0,14 REELS PLUS LLP,BHARAT RASHTRA SAMITHI,10000000
1,A B C INDIA LIMITED,BHARATIYA JANATA PARTY,4000000
2,AAKANKSHA BAHETY,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",1000000
3,AALAYA CONSTRUCTIONS,BHARAT RASHTRA SAMITHI,10000000
4,AARISH SOLAR POWER PRIVATE LIMITED,YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU C...,20000000
...,...,...,...
1653,ZUVAN ENERGY PRIVATE LIMITED,YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU C...,10000000
1654,ZUVAN ENERGY PVT LTD,BHARAT RASHTRA SAMITHI,20000000
1655,ZYDUS HEALTHCARE LIMITED,BHARATIYA JANATA PARTY,180000000
1656,ZYDUS HEALTHCARE LIMITED,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",30000000


In [86]:
filtered_df_groups_second = filtered_df_groups.nlargest(300, "Denominations_x").reset_index().drop(columns=["index"])

In [87]:
filtered_df_groups_second

,Name_of_Purchaser,Name_of_Political_Party,Denominations_x
0,MEGHA ENGINEERING AND INFRASTRUCTURES LI MITED,BHARATIYA JANATA PARTY,5190000000
1,FUTURE GAMING AND HOTEL SERVICES PR,DRAVIDA MUNNETRA KAZHAGAM (DMK),4530000000
2,FUTURE GAMING AND HOTEL SERVICES PR,ALL INDIA TRINAMOOL CONGRESS,4350000000
3,QWIKSUPPLYCHAINPRIVATELIMITED,BHARATIYA JANATA PARTY,3750000000
4,HALDIA ENERGY LIMITED,ALL INDIA TRINAMOOL CONGRESS,2810000000
...,...,...,...
295,PROCURE ADVISORY SERVICES PRIVATE LTD,BIJU JANATA DAL,80000000
296,UNIPRO TECHNO INFRASTRUCTURE PVT LTD,BHARATIYA JANATA PARTY,80000000
297,WESTWELL GASES PRIVATE LIMITED,DRAVIDA MUNNETRA KAZHAGAM (DMK),80000000
298,ZYDUS HEALTHCARE LIMITED,SIKKIM KRANTIKARI MORCHA,80000000


In [90]:
#sankeys to show connections of categories that the projects fall into. 
#This is the most elastic table/code structure we have got. We add the table with a simple structure:
#column 0 (source) | column 1 (target) | column 3 (value/weight)
#this took a lot of work, please conserve this code !!


def sankey(table):
    unique = table.iloc[:,0].unique().tolist()+table.iloc[:,1].unique().tolist()
    #unique labels from both the source and target give us the lables for all the nodes
    
    #next we need to map the numbers/index from the unique list to the actual table becuase plotly sankey only takes indexes
    #for SOURCE we map its index from the UNIQUE list
    #for TARGET we map its index from the UNIQUE list
    
    source = []
    for i in range(table.shape[0]-1):
        x = unique.index(table.iloc[:,0].loc[i])
        source.append(x)
        
    target = []
    for j in range(table.shape[0]-1):
        y = unique.index(table.iloc[:,1].loc[j])
        target.append(y)
    
    #Value will be the weights that we give to the links this will be the third column
    value = table.iloc[:,2].values.tolist()
    
    #sankey construction
    node = dict(label = unique, pad=15, thickness=15)
    link = dict(source = source, target = target, value = value)
    sankey = go.Sankey(link = link, node = node)
    
    #plot
    fig_s = go.Figure(sankey)
    fig_s.update_layout(title_text="Sankey Diagram", font_size=1, width=500, height=1000, margin=dict(l=0, r=0, t=0, b=0))
    
    return fig_s.show()


In [91]:
sankey(filtered_df_groups_second)